# WeebCentral Manga Downloader

**Step 1:** Run the cell below to install Chrome for Testing and clone the repository

In [ ]:
# Install Chrome for Testing (required for Selenium)
!apt-get update -qq
!apt-get install -y -qq libnss3 libatk1.0-0 libatk-bridge2.0-0 libcups2 libxkbcommon0 libxcomposite1 libxdamage1 libxfixes3 libxrandr2 libgbm1 libasound2

# Download Chrome for Testing (stable version)
!wget -q https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.108/linux64/chrome-linux64.zip -O /tmp/chrome.zip
!wget -q https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.108/linux64/chromedriver-linux64.zip -O /tmp/chromedriver.zip

# Extract to /opt
!unzip -q -o /tmp/chrome.zip -d /opt/
!unzip -q -o /tmp/chromedriver.zip -d /opt/
!chmod +x /opt/chromedriver-linux64/chromedriver

print("✅ Chrome for Testing installed!")

# Clone the repository
!rm -rf /content/weebcentral_downloader
!git clone https://github.com/Yui007/weebcentral_downloader

# Install Python dependencies (including undetected-chromedriver for Cloudflare bypass)
!pip install requests beautifulsoup4 selenium tqdm IPython ipywidgets undetected-chromedriver -q

print("✅ Setup complete!")

**Step 2:** Run the manga downloader

In [ ]:
%cd /content/weebcentral_downloader
!python weebcentral_scraper_colab.py

**Step 3 (Optional):** Zip the manga folder for easy download

In [ ]:
import os
import shutil

output_dir = '/content/weebcentral_downloader/manga_downloads'

if os.path.exists(output_dir):
    manga_folders = [d for d in os.listdir(output_dir)
                    if os.path.isdir(os.path.join(output_dir, d))]

    if manga_folders:
        print("Available manga folders:")
        for i, folder in enumerate(manga_folders, 1):
            print(f"{i}. {folder}")

        choice = int(input("\nEnter the number of the folder to zip: ")) - 1
        if 0 <= choice < len(manga_folders):
            folder_name = manga_folders[choice]
            folder_path = os.path.join(output_dir, folder_name)
            zip_path = f"/content/{folder_name}.zip"

            print(f"\nCreating zip file for {folder_name}...")
            shutil.make_archive(f"/content/{folder_name}", 'zip', folder_path)

            print(f"\n✅ Zip file created at: {zip_path}")
            print("Download it from the Colab file browser on the left.")
        else:
            print("Invalid choice!")
    else:
        print("No manga folders found!")
else:
    print(f"Directory {output_dir} not found!")

**Step 4 (Optional):** If the file is too large (>200MB), split into chunks

In [ ]:
import os

output_dir = '/content/weebcentral_downloader/manga_downloads'
SPLIT_SIZE_MB = 200

if os.path.exists(output_dir):
    manga_folders = [d for d in os.listdir(output_dir)
                    if os.path.isdir(os.path.join(output_dir, d))]

    if manga_folders:
        print("Available manga folders:")
        for i, folder in enumerate(manga_folders, 1):
            print(f"{i}. {folder}")

        choice = int(input("\nEnter the number of the folder to split: ")) - 1
        if 0 <= choice < len(manga_folders):
            folder_name = manga_folders[choice]
            folder_path = os.path.join(output_dir, folder_name)

            print(f"\nCreating split archives for {folder_name}...")
            !7z a -v{SPLIT_SIZE_MB}m -mx=1 -mmt=on "/content/{folder_name}.7z" "{folder_path}"/*

            splits = [f for f in os.listdir('/content') if f.startswith(f"{folder_name}.7z")]
            print(f"\n✅ Created {len(splits)} split files at /content/")
            for split in splits:
                size_mb = os.path.getsize(f"/content/{split}") / (1024 * 1024)
                print(f"- {split} ({size_mb:.1f} MB)")
        else:
            print("Invalid choice!")
    else:
        print("No manga folders found!")
else:
    print(f"Directory {output_dir} not found!")